In [1]:
import cv2 as cv
import numpy as np
import pickle
import os
from IPython.display import clear_output
from PIL import Image

#### Loading cascade files

In [2]:
face_classifier = cv.CascadeClassifier('./Cascade/haarcascade_frontalface_default.xml')
lips_classifier = cv.CascadeClassifier('./Cascade/haarcascade_smile.xml')
left_eye_classifier = cv.CascadeClassifier('./Cascade/haarcascade_lefteye_2splits.xml')
right_eye_classifier = cv.CascadeClassifier('./Cascade/haarcascade_righteye_2splits.xml')

#### Loading models

In [3]:
def sigmoid_function(lst):
    output = []
    for i in lst:
        if i[1] >= 0.5:
            output.append(1)
        else:
            output.append(0)
    return output

In [4]:
with open('eye_state_classification_model.pkl','rb') as f:
    eye_classifier_model = pickle.load(f)

In [5]:
right_eye_img = cv.imread('./Dataset/Test_images/right_eye.png')
right_eye_img = cv.resize(right_eye_img, (70,70))

In [6]:
state = sigmoid_function(eye_classifier_model.predict(np.array([right_eye_img])))

print(str(state[0]))

1/1 [==============================] - 1s 851ms/step
1


#### starting video cam

In [16]:
cam = cv.VideoCapture(0)
left_state = "Closed"
right_state = "Closed"
img_idx = 1

while True:
    clear_output(wait=True)
    _,img = cam.read()
    img = cv.flip(img,1)

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # face detection
    try:
        faces = face_classifier.detectMultiScale(gray, 1.1, 5)

        for x,y,w,h in faces:
            cv.putText(img, "Face Detected ", (30,30),cv.FONT_HERSHEY_PLAIN, 2, (0,180,0), 2)
            cv.rectangle(img, (x,y), (x+w, y+h), (0,180,0), 2)
    except:
        pass

    # smile detection
    # try:
    #     lips = lips_classifier.detectMultiScale(gray, 1.1, 200)

    #     for x,y,w,h in lips:
    #         cv.rectangle(img, (x,y), (x+w, y+h), (0,180,0), 1)
    # except:
    #     pass

    # left eye detection
    try:
        left_eyes = left_eye_classifier.detectMultiScale(gray, 1.1, 5)

        for x,y,w,h in left_eyes:
            cv.rectangle(img, (x,y), (x+w, y+h), (0,180,0), 1)
            cv.putText(img, left_state, (30,60),cv.FONT_HERSHEY_PLAIN, 2, (0,180,0), 2)
            left_eye_img = img[y:y+h, x:x+w]

        left_eye_img = cv.resize(left_eye_img, (70,70))
        state = sigmoid_function(eye_classifier_model.predict(np.array([left_eye_img])))
        left_state = "Left eye is open" if state[0]==1 else "Left eye is close"
    except Exception as e:
        print(e)
        pass

    # right eye detection
    try:
        right_eyes = right_eye_classifier.detectMultiScale(gray, 1.1, 5)

        for x,y,w,h in right_eyes:
            cv.rectangle(img, (x,y), (x+w, y+h), (0,180,0), 1)
            cv.putText(img, right_state, (30,90),cv.FONT_HERSHEY_PLAIN, 2, (0,180,0), 2)
            right_eye_img = img[y:y+h, x:x+w]

        right_eye_img = cv.resize(right_eye_img, (70,70))
        state = sigmoid_function(eye_classifier_model.predict(np.array([right_eye_img])))
        right_state = "Right eye is open" if state[0]==1 else "Right eye is close"
    except:
        pass

    cv.imshow("Frame",img)
    cv.imshow('BW',gray)

    if cv.waitKey(10) == ord('c'):
        cv.imwrite(f'result{img_idx + 1}.png',img)
        img_idx += 1

    if cv.waitKey(10) == 27:
        break

cam.release()
cv.destroyAllWindows()

1/1 [==============================] - 0s 53ms/step
